In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
DATA_DIR="../input/petfinder-pawpularity-score/"

In [3]:
train_meta_df=pd.read_csv(DATA_DIR+"train.csv")
test_meta_df=pd.read_csv(DATA_DIR+"test.csv")

print("Train Metadata shape:", train_meta_df.shape)
print("Test Metadata Shape:", test_meta_df.shape)

In [4]:
train_meta_df.head()

In [5]:
test_meta_df.head()

In [6]:
X=train_meta_df.drop(columns=["Id","Pawpularity"])
y=train_meta_df["Pawpularity"]

#Copy training data
train_meta_df_pred=train_meta_df.copy()

In [7]:
#Import library for modeling

from sklearn.model_selection import train_test_split, GridSearchCV,KFold
from sklearn.metrics import mean_squared_error

Building Regression Models

In [8]:
import lightgbm as lgbm

In [9]:
import warnings
warnings.filterwarnings("ignore")

In [10]:
#パラメータ調節
def param():
    param_grid= {
        'metric':['rmse'],
        'n_estimators':[5,10,25,50,100],
        'num_leaves': [5,10,15,30],
        "max_depth":[5,10,50,100],
        'learning_rate': [0.01,0.05,0.1]
    }
    
    return param_grid


model=lgbm.LGBMRegressor(
    random_state=42,
    verbose=0
)


grid_search = GridSearchCV(
    model,
    param(),
    cv=KFold(n_splits=3),
    n_jobs=-1
)

#grid_search.fit(X,y)
#print(grid_search.best_estimator_)
#print(grid_search.best_score_)

In [11]:
best_model=lgbm.LGBMRegressor(max_depth=5,n_estimators=10,num_leaves=5,random_state=42,verbose=0)

In [12]:
#LightGBM prediction
best_model.fit(X,y)

In [13]:
test_y_pred=best_model.predict(test_meta_df.drop(columns=["Id"]))

In [14]:
submission=pd.read_csv(DATA_DIR+"sample_submission.csv")


In [15]:
submission["Id"]=test_meta_df["Id"]
submission["Pawpularity"]=test_y_pred

submission.head(5)

In [16]:
submission.to_csv("submission.csv",index=False)